<a href="https://colab.research.google.com/github/thatsciencegal/flood_pheno/blob/main/SASA_PEAN_cc_rem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialize Google Earth Engine

In [ ]:
import ee

## Trigger the authentication flow
ee.Authenticate()

## Initialize the library
ee.Initialize()

## Test the API by printing elevation of Mt. Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Add mapping abilities

In [24]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

Import the Landsat 8 data for the area between São Salvador and Peixe Angical

In [73]:
## Import the Landsat 8 data for the area between São Salvador and Peixe Angical
## Import only the data with < 40% cloud cover

l8SasaPean = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
          .filter(ee.Filter.eq('WRS_PATH', 222))\
          .filter(ee.Filter.eq('WRS_ROW', 69))\
          .filterDate('2017-07-01', '2018-06-30')\
          .filterMetadata('CLOUD_COVER', 'less_than', 40)

count = l8SasaPean.size()
print('Image count:', count.getInfo())

## Get sun angle info
sunAz = l8SasaPean.aggregate_array('SOLAR_AZIMUTH_ANGLE')
print(sunAz.getInfo())

sunZen = l8SasaPean.aggregate_array('SOLAR_ZENITH_ANGLE')
print(sunZen.getInfo())

## Change list into dataframe
from pandas import DataFrame
import pandas as pd

sunAzDf = DataFrame(sunAz.getInfo(), columns=['solar_azimuth'])
sunZenDf = DataFrame(sunZen.getInfo(), columns=['solar_zenith'])

frames = [sunAzDf, sunZenDf]

sunAngles = pd.concat(frames, axis=1)
print(sunAngles)



Image count: 13
[41.201656, 44.603764, 49.145565, 55.083778, 62.930813, 86.018692, 109.071152, 106.020981, 63.799332, 46.45969, 41.515007, 38.957207, 38.370541]
[45.534126, 43.23679, 39.761745, 35.508755, 31.049667, 24.623917, 25.051178, 31.107407, 34.019474, 39.332912, 42.298904, 44.779724, 46.276974]
    solar_azimuth
0       41.201656
1       44.603764
2       49.145565
3       55.083778
4       62.930813
5       86.018692
6      109.071152
7      106.020981
8       63.799332
9       46.459690
10      41.515007
11      38.957207
12      38.370541
    solar_zenith
0      45.534126
1      43.236790
2      39.761745
3      35.508755
4      31.049667
5      24.623917
6      25.051178
7      31.107407
8      34.019474
9      39.332912
10     42.298904
11     44.779724
12     46.276974
    solar_azimuth  solar_zenith
0       41.201656     45.534126
1       44.603764     43.236790
2       49.145565     39.761745
3       55.083778     35.508755
4       62.930813     31.049667
5       86.018